In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from tqdm import tqdm
from collections import deque
import wave, struct
import cProfile, re
#cProfile.run('re.compile("foo|bar")')

In [11]:
WIDTH = int(720)

def normilizeFrame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite("gray.png", frame)
    frame = cv2.resize(frame, (WIDTH, frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    return frame / 255.0

fileName = 'testfiles/amaz_horse'

cap = cv2.VideoCapture(fileName + '.avi')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret, frame = cap.read()
if not ret:
    print('error reading frame')

gray = normilizeFrame(frame)

In [3]:
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height)
print(frameCount)

720.0
625.0
1986.0


In [4]:
model = keras.models.load_model("model")

In [5]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 720)]        0           []                               
                                                                                                  
 layer_1 (Dense)                (None, 128)          92288       ['input[0][0]']                  
                                                                                                  
 layer_2 (Dense)                (None, 128)          16512       ['layer_1[0][0]']                
                                                                                                  
 binaryData84 (Dense)           (None, 84)           10836       ['layer_2[0][0]']                
                                                                                            

In [6]:
def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return np.concatenate((even_fields, odd_fields), axis=0)

bufL0 = deque()
bufR0 = deque()
bufL1 = deque()
bufR1 = deque()
bufL2 = deque()

#kluge init queues
for i in range((16*5)-1):
    bufL0.append(0)

for i in range((16*4)-1):
    bufR0.append(0)

for i in range((16*3)-1):
    bufL1.append(0)

for i in range((16*2)-1):
    bufR1.append(0)
    
for i in range(16-1):
    bufL2.append(0)

def deinterlive(block):
    
    bufL0.append(int(block[0]))
    bufR0.append(int(block[1]))
    bufL1.append(int(block[2]))
    bufR1.append(int(block[3]))
    bufL2.append(int(block[4]))
    
    return bufL0.popleft(), bufR0.popleft(), bufL1.popleft(), bufR1.popleft(), bufL2.popleft(), int(block[5]) #L0, R0, L1, R1, L2, R2

In [19]:
bar = tqdm(total=frameCount - (cap.get(cv2.CAP_PROP_POS_FRAMES) -1))
bar.set_description("decoding")
file = wave.open(fileName + '.wav', mode='w')
file.setnchannels(2) 
file.setsampwidth(2)
file.setframerate(44100)

decoding:   0%|          | 0/1202.0 [00:02<?, ?it/s]
decoding:   0%|          | 0/1202.0 [00:00<?, ?it/s]

In [13]:
def a():
    while True:
        bar.update(1)
        if cap.grab():
            ret, frame = cap.retrieve()
            
            if not ret:
                print('error reading frame')
                break
                    
            gray = normilizeFrame(frame)
            #cv2.imwrite("gray_norm.png", frame)
            
            gray = deinterlase(gray)
            
            #cv2.imwrite("gray_deint.png", gray)
            service, data = model.predict(gray, verbose=0, batch_size=1)
            service = service > 0.5
            
            audioFrame = bytearray()
            
            for i in range(len(service)):
                if service[i][0]:
                    L0, R0, L1, R1, L2, R2 = deinterlive(data[i])
                    audioFrame += struct.pack('<H', (L0 << 2))
                    audioFrame += struct.pack('<H', (R0 << 2))
                    audioFrame += struct.pack('<H', (L1 << 2))
                    audioFrame += struct.pack('<H', (R1 << 2))
                    audioFrame += struct.pack('<H', (L2 << 2))
                    audioFrame += struct.pack('<H', (R2 << 2))
            
            keras.backend.clear_session()
            file.writeframes(audioFrame)
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == frameCount:
            break

In [20]:
cProfile.run('a()')

decoding:   6%|▌         | 69/1202.0 [00:20<03:43,  5.08it/s] 

         3204228 function calls (3064941 primitive calls) in 13.746 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       69    0.234    0.003    0.263    0.004 1770470628.py:3(normilizeFrame)
        1    0.149    0.149   13.748   13.748 2553216932.py:1(a)
       69    0.002    0.000    0.181    0.003 3958513488.py:1(deinterlase)
    40710    0.165    0.000    0.217    0.000 3958513488.py:28(deinterlive)
       69    0.001    0.000    0.179    0.003 <__array_function__ internals>:177(concatenate)
      138    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(ndim)
  414/207    0.005    0.000    0.287    0.001 <frozen importlib._bootstrap>:1002(_find_and_load)
      207    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
      414    0.002    0.000    0.002    0.000 <frozen importlib._bootstrap>:112(release)
      414    0.000    0.000    0.000    0.000 <frozen importl

KeyboardInterrupt: 

decoding:   6%|▌         | 69/1202.0 [00:34<09:24,  2.01it/s]